# Projectile Motion with Air Resistance

Starting with Newton's Law,

$m\dot{\vec{v}} = \sum \vec{F} = \vec{D}+\vec{G}$

$m\dot{\vec{v}} = \frac{-\pi R^2 \rho C}{2}\frac{\dot{x}(\dot{x}^2+\dot{y}^2)}{\sqrt{\dot{x}^2+\dot{y}^2}}\hat{x}-\left[\frac{\pi R^2 \rho C}{2}\frac{\dot{y}(\dot{x}^2+\dot{y}^2)}{\sqrt{\dot{x}^2+\dot{y}^2}}\hat{x}+mg\right]\hat{y}$

$\dot{\vec{v}}= \frac{-\pi R^2 \rho C}{2m}\frac{\dot{x}(\dot{x}^2+\dot{y}^2)\sqrt{\dot{x}^2+\dot{y}^2}}{\dot{x}^2+\dot{y}^2}\hat{x} - \left[\frac{\pi R^2 \rho C}{2m}\frac{\dot{y}(\dot{x}^2+\dot{y}^2)\sqrt{\dot{x}^2+\dot{y}^2}}{\dot{x}^2+\dot{y}^2}+g\right]\hat{y}=\frac{-\pi R^2 \rho C}{2m}\dot{x}\sqrt{\dot{x}^2+\dot{y}^2}\hat{x}- \left[\frac{\pi R^2 \rho C}{2m}\dot{y}\sqrt{\dot{x}^2+\dot{y}^2}+g\right]\hat{y}$

where $\vec{D}=\frac{-\pi R^2\rho C^2 |\vec{v}|^2}{2}\hat{v}=\frac{-\pi R^2\rho C^2}{2}(\dot{x}^2+\dot{y}^2)\frac{\dot{x}\hat{x}+\dot{y}\hat{y}}{\sqrt{\dot{x}^2+\dot{y}^2}}$.

Let $\vec{v}=\dot{x}\hat{x}+\dot{y}\hat{y}$ \& $\textrm{\textbf{D}}=\frac{-\pi R^2 \rho C}{2m}$,

$\ddot{x}=\textrm{\textbf{D}}\dot{x}\sqrt{\dot{x}^2+\dot{y}^2}$ & $\ddot{y}=\textrm{\textbf{D}}\dot{y}\sqrt{\dot{x}^2+\dot{y}^2}-g$

Let $\alpha = \dot{x}$ & $\beta = \dot{y} \rightarrow \dot{\alpha}=\textrm{\textbf{D}}\alpha\sqrt{\alpha^2 + \beta^2}$ & $\dot{\beta}=\textrm{\textbf{D}}\beta\sqrt{\alpha^2 + \beta^2} -g$

Thus, the projectile can be modeled through the following set of $1^{\textrm{st}}$ Order ODE: $\lbrace \alpha = \dot{x}, \dot{\alpha}=\textrm{\textbf{D}}\alpha\sqrt{\alpha^2 + \beta^2}, \beta = \dot{y}, \dot{\beta}=\textrm{\textbf{D}}\beta\sqrt{\alpha^2 + \beta^2} -g\rbrace$
